# Lambda distribution (Vs Reff)

In [1]:
import matplotlib.pyplot as plt
import pickle
import numpy as np

## fucntions 
def load_pickle(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)


def plot_lambda(catalog, i_early, i_late, i_bad, fn_out='./'):
    import matplotlib.pyplot as plt
    plt.ioff()
    f = plt.figure()
    ax = f.add_subplot(111)
    #for i, val in enumerate(lambdar_arr):
    for i in i_early:
        a = np.asarray(catalog['lambda_arr'][i])
        ax.plot(a, 'r-', alpha=0.5) # Red = Early
    for i in i_late:
        ax.plot(catalog['lambda_arr'][i], 'b-', alpha=0.3) # Red = Early
    
    #plt.xlabel() # in the unit of Reff
    ax.set_title(r"$\lambda _{R}$") 
    ax.set_ylabel(r"$\lambda _{R}$") 
    ax.set_xlabel("["+ r'$R/R_{eff}$'+"]")
    ax.set_xlim(right=9)
    ax.set_ylim([0,1])
    ax.set_xticks([0, 4.5, 9])
    ax.set_xticklabels(["0", "0.5", "1"])
    plt.savefig(fn_out)
    plt.close()
    
def aexp2zred(aexp):
    return [1.0/a - 1.0 for a in aexp]

def zred2aexp(zred):
    return [1.0/(1.0 + z) for z in zred]

def lbt2aexp(lts):
    import astropy.units as u
    from astropy.cosmology import WMAP7, z_at_value
    zreds = [z_at_value(WMAP7.lookback_time, ll * u.Gyr) for ll in lts]
    return [1.0/(1+z) for z in zreds]

def density_map(x, y, sort=True):
    from scipy.stats import gaussian_kde
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy) 
    z /= max(z)

    idx = z.argsort()    
    xx, yy = x[idx], y[idx]
    z = z[idx]
    
    return xx, yy, z

##  I like this!

In [2]:
clusters = ['05427', '36415', '39990', '01605', '10002', '36413', '04466', '74010'][0:5]

lr_points = 10 # number of points int 1 Reff.
nreff = 3
nbins = 20

def lambda_den_map(clusters, nout=187, lr_points = 10, nreff=3, nbins=20):
    print(" nout:", nout, "lr_points:", lr_points, "nreff:", nreff, "nbins:", nbins)
    
    points = np.arange(lr_points * nreff)
    x_ticks_label = ["0", "1", "2", "3", "4"][0:nreff]

    x_tick_pos = [0]
    [x_tick_pos.append((i+1)*lr_points) for i in range(nreff)]

    # Need a compiled array of lambda_arr
    fig, axs = plt.subplots(2,2, sharey=True)#, sharex=True)

    mass_cut_l = [0, 5e9, 1e10, 1e11, 1e12]
    mass_cut_r = [1e13,1e10, 1e11, 1e12, 1e13]

    yticks_ok=[0.0, 0.2, 0.4, 0.6, 0.8]
    lambda_range=[0.0, 0.8]

    snout = str(nout)
    for imass in range(4):

        # Count number of galaxies
        ngood=0
        for i, cluster in enumerate(clusters):
            wdir = '/home/hoseung/Work/data/' + cluster
            catalog = load_pickle(wdir + '/catalog_GM/' + 'catalog' + snout + '.pickle')
            i_good = np.where((catalog['mstar'] > mass_cut_l[imass]) & (catalog['mstar'] < mass_cut_r[imass]))[0]
            ngood += len(i_good)

        ax = axs.ravel()[imass]
        all_lr = np.zeros((len(points), ngood))

        ngood=0
        for i, cluster in enumerate(clusters):
            wdir = '/home/hoseung/Work/data/' + cluster
            catalog = load_pickle(wdir + '/catalog_GM/' + 'catalog' + snout + '.pickle')
            i_good = np.where((catalog['mstar'] > mass_cut_l[imass]) & (catalog['mstar'] < mass_cut_r[imass]))[0]

            for i, i_gal in enumerate(i_good):
                all_lr[:,ngood + i] = catalog['lambda_arr'][i_gal][:len(points)]
            ngood +=len(i_good)

        # Draw density maps   
        den_map = np.zeros((nbins, len(points)))
        for i in range(len(points)):
            den_map[:,i], ypoints = np.histogram(all_lr[i,:], bins=nbins, range=lambda_range)

        den_map /= den_map.max()
        im = ax.imshow(den_map, origin="lower", cmap="Blues", interpolation="none")

        #ax.set_xlim([-0.5,2*lr_points])
        ax.set_ylim([-0.5,nbins]) 
        ax.set_title(r"{:.1e} $< M_\ast <$ {:.1e}".format(mass_cut_l[imass], mass_cut_r[imass]))
        ax.text(3,17, "# gals:" + str(ngood)) # data coordinates

        #ax.set_yticks([np.where(ypoints == yy)[0] for yy in [0.0, 0.2, 0.4, 0.6, 0.8]]) # 0.0, 0.2, 0.4, 0.6, 0.8
        #ax.set_yticklabels([str(yy) for yy in yticks_ok])

    for j in range(2):
        for i in range(2):
            axs[j,i].set_xticks(x_tick_pos)
            axs[j,i].set_xticklabels(labels = [z for z in x_ticks_label])
            axs[1,i].set_xlabel(r"$R/R_{eff}$")

            #axs[i,0].set_ylabel("$\lambda _R$")
            axs[i,j].set_yticks([np.where(ypoints == yy)[0] for yy in np.arange(lambda_range[0], lambda_range[1])]) # 0.0, 0.2, 0.4, 0.6, 0.8
            axs[i,j].set_yticklabels([str(yy) for yy in yticks_ok])

    # Add + mark at 0.5, 1.0, 2.0Reff
    #fig.tight_layout()
    cax = fig.add_axes([0.86, 0.1, 0.03, 0.8]) # [left corner x, left corner y, x width, y width]
    plt.colorbar(im, cax=cax, label='normalized denisty')

    plt.subplots_adjust(left=0.1, bottom=None, right=0.8, top=None, wspace=0.05, hspace=0.22)

    #left  = 0.125  # the left side of the subplots of the figure
    #right = 0.9    # the right side of the subplots of the figure
    #bottom = 0.1   # the bottom of the subplots of the figure
    #top = 0.9      # the top of the subplots of the figure
    #wspace = 0.2   # the amount of width reserved for blank space between subplots
    #hspace = 0.5   # the amount of height reserved for white space between subplots

    plt.show()
    

In [3]:
#lambda_den_map(clusters)
lambda_den_map(["01605_hi"], nout=368)

 nout: 368 lr_points: 10 nreff: 3 nbins: 20


High resolution run seems to have more galaxies. 
check mass function. 

In [4]:
clusters = [5427, 36415, 39990, 1605, 10002, 36413, 4466, 74010][0:5]
# 74010 is unreliable.
# 36413 왜 안 돌아가나..?

#exclude_gals = [[],
#                [],
#                [],
#                [123,155,],
#                [2694,4684,5448,5885,5906,6967,6981,7047,7072,7151,7612],
#                []]

lr_points = 10 # number of points int 1 Reff.
nreff = 3
points = np.arange(lr_points * nreff)
x_ticks_label = ["0", "1", "2", "3", "4"][0:nreff]

x_tick_pos = [0]
[x_tick_pos.append((i+1)*lr_points) for i in range(nreff)]

# Need a compiled array of lambda_arr
fig, axs = plt.subplots(2,2, sharey=True, sharex=True)

mass_cut_l = [0, 5e9, 1e10, 1e11, 1e12]
mass_cut_r = [1e13,1e10, 1e11, 1e12, 1e13]
#titles = #["All galaxies from all clusters", 
#          " {} $< M_{*} <$ {}".format(mass_cut_l[imass], mass_cut_r[imass])]


for imass in range(4):
    ax = axs.ravel()[imass]
    all_lr = np.zeros(0)
    xpos = [] # why list?? 
    ypos = []
    zpos = []
    clur = []

    for i, cluster in enumerate(clusters):
        wdir = '/home/hoseung/Work/data/' + str(cluster).zfill(5)
        catalog = load_pickle(wdir + '/catalog_GM/' + 'catalog187.pickle')
        i_good = np.where((catalog['mstar'] > mass_cut_l[imass]) & (catalog['mstar'] < mass_cut_r[imass]))[0]

        for ij, j in enumerate(i_good):
            #catalog['lambda_arr'][#galaxies][#number of points]
            all_lr = np.concatenate((all_lr, catalog['lambda_arr'][j][:len(points)])) # some catalog has L_arr up to 2Rvir. 
    
    # Draw density maps
    
    # x values
    xpoints = np.tile(points, len(all_lr))
    # Gaussian_kde measures 2D density. But in this case x-axis and y-axis are two independent parameters 
    #(not like x position and y position). So instead, measure 1-D histogram at each x point (R/Reff).

    xx, yy, z = density_map(xpoints[:all_lr.shape[0]], all_lr.ravel(), ax)
    ax.scatter(xx, yy, c=z, s=50, edgecolor='')
    ax.set_xlim([-0.5,2*lr_points])
    ax.set_ylim([0,0.8])
    ax.set_title(r"{:.1e} $< M_\ast <$ {:.1e}".format(mass_cut_l[imass], mass_cut_r[imass]))

    
axs[1,0].set_xticks(x_tick_pos)#[0,lr_points-1,2*lr_points - 1])
axs[1,0].set_xticklabels(labels = [z for z in x_ticks_label])
axs[1,0].set_xlabel(r"$R/R_{eff}$")
axs[1,1].set_xticks(x_tick_pos)#[0,lr_points-1,2*lr_points - 1])
axs[1,1].set_xticklabels(labels = [z for z in x_ticks_label])
axs[1,1].set_xlabel(r"$R/R_{eff}$")


axs[0,0].set_ylabel("$\lambda _R$")
axs[1,0].set_ylabel("$\lambda _R$")

# Add + mark at 0.5, 1.0, 2.0Reff


plt.show()
    

In [ ]:
plt.close()

####  Seaborn heat map 
looks better than imshow. (BTW, you can use pcolomesh (X,Y,Z) instead of imshow(map))
Choose a suitable color map from Seaborn color map templates.

In [8]:
#plt.clf()
fig, ax = plt.subplots(1)
import seaborn.apionly as sns

# reset rc params to defaults
sns.reset_orig()
#cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(den_map, cmap="Blues", square=True, xticklabels=5, yticklabels=5,
            linewidths=.2, cbar_kws={"shrink": .5}, ax=ax)
plt.gca().invert_yaxis()
plt.show()

In [5]:
# as a line
plt.close()

fig, ax = plt.subplots(len(clusters))

for i, cluster in enumerate(clusters):
    wdir = '/home/hoseung/Work/data/' + str(cluster).zfill(5)
    catalog = load_pickle(wdir + '/catalog_GM/' + 'catalog187.pickle')
    #i_early = np.where(catalog['mstar'] > 5e11)[0]
    i_early = np.where((catalog['mstar'] > 1e10) & (catalog['mstar'] < 1e11))[0]
    
    for j in i_early:
        ax[i].plot(points, catalog['lambda_arr'][j][:2 *lr_points], c='grey', alpha=0.3)
    
    ax[i].set_xlim([-0.5,2*lr_points])
    ax[i].set_ylim([0,0.8])
    x_tick_pos = ""
    ax[i].set_xticks([0,lr_points -1, 2*lr_points - 1])
    x_ticks_label = ["0", "1", "2"] # Correct. by default, rscale_lambda = 2.0
    ax[i].set_xticklabels(labels = [z for z in x_ticks_label])
    ax[i].set_xlabel(r"$R/R_{eff}$")
    
plt.show()

ValueError: x and y must have same first dimension

In [11]:
len(catalog['lambda_arr'][j])

45

In [ ]:
final_gals = list(cat['final_gal'])

    # exclude disky galaxies
for bad_gal in exclude_gals[i]:
    final_gals.remove(bad_gal)

ngals = len(final_gals)
mstar = np.zeros((ngals, nnouts))
l_r = np.zeros((ngals, nnouts))